Чат-бот
Докрутить чат-бота (обучить лучший ft, w2v, взвесить idf). Модели сдампить и сложить в облако, ноутбук можно через гитхаб. Ответы: https://yadi.sk/d/SLgpFrmrBubeYg

In [1]:
# !pip install pymorphy2
# !pip install stop_words
# !pip install annoy

In [2]:
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# assert False

#Small preprocess of the answers

question = None
written = False

with open("/content/prepared_answers.txt", "w") as fout:
    # with open("/content/drive/My Drive/Otvety.txt", "r") as fin:
    with open("/content/drive/My Drive/NLP/HW2/otvety.txt", "r") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


In [5]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [6]:
sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

# with open("/content/drive/My Drive/Otvety.txt", "r") as fin:
with open("/content/drive/My Drive/NLP/HW2/otvety.txt", "r") as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 100000:
            break

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


In [7]:
sentences = [i for i in sentences if len(i) > 2]

In [8]:
sentences[0]

['вопрос', 'тдв', 'отдыхать', 'лично', 'советовать', 'завести']

In [9]:
modelFT = FastText(sentences=sentences, size=200, min_count=2, window=5)

In [10]:
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.feature_extraction.text import TfidfVectorizer

    def tokenizer(text):
      return text

In [11]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=tokenizer,
                                   lowercase=False, max_df=0.9)

In [12]:
tfidf_vectorizer.fit_transform(sentences)


<64662x165629 sparse matrix of type '<class 'numpy.float64'>'
	with 946117 stored elements in Compressed Sparse Row format>

In [13]:
tfidf_vectorizer.idf_[tfidf_vectorizer.vocabulary_['вопрос']]

4.487608623568247

In [15]:
import warnings
warnings.filterwarnings("ignore")

In [16]:
ft_index = annoy.AnnoyIndex(200 ,'angular')

index_map = {}
counter = 0

with open("/content/prepared_answers.txt", "r") as f:
    for line in tqdm_notebook(f):
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        
        vector_ft = np.zeros(200)
        for word in question:
            if word in modelFT:
                vector_ft += modelFT[word]
                n_ft += tfidf_vectorizer.idf_[tfidf_vectorizer.vocabulary_[word]] if word in tfidf_vectorizer.vocabulary_ else 1
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index.add_item(counter, vector_ft)
            
        counter += 1
        
        if counter > 100000:
            break

ft_index.build(50)

True

In [17]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(200)
    norm = 0
    for word in question:
        if word in model:
            vector += model[word]
            norm += tfidf_vectorizer.idf_[tfidf_vectorizer.vocabulary_[word]]
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 3)
    return [index_map[i] for i in answers]

In [18]:
TEXT = "что такое счастье"

In [19]:
get_response(TEXT, ft_index, modelFT, index_map)

['Это когда у тя в жизни всё в поряде. \n',
 'Просыпаешся утром.Голова болит и настроение казалось бы х....Думаешь: "Че я здесь делаю, в этом городе, где нет возможностей и свободы, надо рвануть куда-н., быть может где-то меня ждет удача,,быть может я найду свою любовь...." Начинаешь вспоминать прожитые дни, прекрасные минуты жизни, друзей бесшабашных, хороших и чертовски красивых подруг, родных, и на душе становится просто КЛ.Идешь на кухню, открываешь холодильник и достаешь оттуда упаковку хорошего прохладного сока, выпиваешь и уже ничего не хочется.Забываешь про дурные мысли и продолжаешь жить.ВОТ ОНО СЧАСТЬЕ!!!. \n',
 'посмотри в зеркало, вот оно. \n']